# Architecture draft 

candidat 1   : prompt, colère

candidat 2   : prompt, colère

présentateur : prompt, carte

audience: (a, b, prompt) -> audimat

le jeu s'arrête si un des colères = 1 ou si audimat = 0

In [1]:
import numpy as np

In [40]:
import numpy as np

In [125]:
import numpy as np

def calculate_score(x, y, upper_bound=10, lower_bound=2):
    """
    Calculate interest score based on anger levels of two debators.
    """
    # Combine x and y to compute a score
    combined_value = (x + y) / 2  # Average the two inputs
    
    # Scale the combined value to the range [lower_bound, upper_bound]
    score = lower_bound + combined_value * (upper_bound - lower_bound)
    
    return int(score)

class Debator():
    def __init__(self,
                 background):
        """
        Initializes a debator with anger level and decay rate.
        background is the biography and mood summarizes the 
        current state of the debator in the debate
        """
        self.anger = 0
        self.anger_decay = 0.05  # Higher decay rate reduces instability

        self.background=background
        self.mood=None

    def speaks(self):
        """
        given the background of the  of the debator
        """

    def 

    def update_anger(self, reaction_score):
        """
        Updates the anger level based on the reaction score.
        """
        new_anger = max(0, self.anger + reaction_score - self.anger_decay)
        self.anger = new_anger

    def quit(self):
        """
        Determines if the debator leaves the debate due to high anger.
        """
        return self.anger >= 1.0

class Debate():
    def __init__(self):
        """
        Initializes the debate with an initial audimat.
        """
        self.audimat = 100

    def update_audimat(self, score):
        """
        Updates the audimat based on the anger levels and random quitting.
        """
        # Simulate a small, random decrease in viewers each round
        quitters = np.random.randint(0, 5)  # Fewer random quitters

        # Update audimat with influence of debator anger scores
        update = self.audimat + score - quitters
        self.audimat = update

    def end(self):
        """
        Ends the debate if the audimat falls below a threshold.
        """
        return self.audimat <= 0


class Presenter():
    def __init__(self,
                 debate_context):
        pass

    def intervention(user_prompt=None):
        """
        takes as input the prompt of the user
        if the prompt is none (nothing said, the debates continues)

        orht
        """
        if user_prompt is None:
            return None

        else:
            processed_prompt=f(user_prompt) # here we design a function that
                                         # will call the LLM to generate a prompt
                                         # given the context etc 
            return processed_prompt



# Initialize debators and debate
trump = Debator()
putin = Debator()
presenter=Presenter()


debate = Debate()

print('Trump initial anger level:', trump.anger)
print('Putin initial anger level:', putin.anger)


Trump initial anger level: 0
Putin initial anger level: 0
The debate was too boring and everybody left.
Number of steps elapsed: 50992


In [126]:


presenter_prompt=None
# Simulate the debate
for i in range(100000):

    # Trump says something
    trump_prompt=trump.speaks(presenter_prompt)

    # Putin reacts and says something
    putin_reaction=putin.digests(trump_prompt)
    putin_prompt=putin.speaks(trump_prompt)

    # update of the anger score of
    # each 
    trump_reaction=trump.digest(putin_prompt)

    trump.update_anger(trump_reaction)
    putin.update_anger(putin_reaction)


    # The presenter intervenes
    presenter_promt=presenter.intervention()
    
    # Generate reaction scores for both debators
    trump_reaction =  0.#np.random.uniform(0.01, 0.1)  # Control anger increments
    putin_reaction = 0.#np.random.uniform(0.01, 0.1)

    #print(f"Trump anger: {trump.anger:.2f}, Putin anger: {putin.anger:.2f}")

    # Calculate interest score
    increased_viewers = calculate_score(trump.anger, putin.anger)

    # Update the audimat
    debate.update_audimat(increased_viewers)

    # Check if the debate should end
    if trump.quit() or putin.quit():
        print("One of the debators was too angry and left the debate.")
        print("Audimat when the debator left: {}.".format(debate.audimat))
        print('Number of steps elapsed: {}'.format(i))
        break

    if debate.end():
        print("The debate was too boring and everybody left.")
        print('Number of steps elapsed: {}'.format(i))
        break

Trump initial anger level: 0
Poutin initial anger level: 0
0.06881210721481278
0.11880841227478604
100
0.8826484973217018
0.534724340010684
107
1.0881036459500475
1.392747186882833
118
One of the debator was too angry and left the debate.


In [14]:
import numpy as np

def generate_state_matrix(n, init_type='uniform', random_seed=None):
    """
    Generates a state (or transition) matrix.

    Parameters:
    - n (int): Number of states (size of the matrix will be n x n).
    - init_type (str): Type of initialization:
        - 'zeros': All transitions start at 0.
        - 'random': Random values between 0 and 1.
        - 'uniform': Uniform probabilities (all rows sum to 1).
    - random_seed (int, optional): Seed for reproducible random matrices.

    Returns:
    - np.ndarray: The generated state matrix.
    """
    if random_seed is not None:
        np.random.seed(random_seed)

    if init_type == 'zeros':
        # All values initialized to 0
        matrix = np.zeros((n, n))
    elif init_type == 'random':
        # Random values between 0 and 1
        matrix = np.random.rand(n, n)
    elif init_type == 'uniform':
        # Uniform probabilities for each row
        matrix = np.random.rand(n, n)
        matrix = matrix / matrix.sum(axis=1, keepdims=True)  # Normalize rows to sum to 1
    else:
        raise ValueError(f"Invalid init_type '{init_type}'. Use 'zeros', 'random', or 'uniform'.")
    
    return matrix

def update_transition_matrix(transition_matrix, current_state_index, action_scores):
    """
    Updates the transition matrix by adding action scores to the current state's row.
    Optionally normalizes the row to maintain valid transition probabilities.

    Parameters:
    - transition_matrix (np.ndarray): The transition matrix (n x n).
    - current_state_index (int): The index of the current state (row).
    - action_scores (np.ndarray): The action scores for all states (n, values can be unnormalized).

    Returns:
    - np.ndarray: The updated transition matrix.
    """
    # Ensure the matrices and vectors have the correct shape
    if action_scores.shape[0] != transition_matrix.shape[1]:
        raise ValueError("Action scores must have the same length as the number of states.")
    
    # Add the action scores to the current state row
    transition_matrix[current_state_index] += action_scores

    # Optional: Normalize the row so that it sums to 1 (if the values represent probabilities)
    row_sum = transition_matrix[current_state_index].sum()
    if row_sum != 0:
        transition_matrix[current_state_index] /= row_sum  # Normalize the row

    return transition_matrix

class Architecture():
    def __init__(self,
                 n_states,
                 initial_state,
                 final_state,
                 random_seed=42
                 ):
        """
        n_states the number of states to consider
        initial_state: the index of the initial state in which 
                       the game starts
        final state: the index of the state in which the game ends if the 
                    player is in this state for two indices in a row
        """

        self.n_states=n_states
        self.initial_state=initial_state

        self.final_state=final_state # is the state in which the action ends.
        self.end=False # will be the end condition 

        # initialize a list that keep track of the states
        # of the agent
        self.states_list=[initial_state]

        # create a transition matrix between the states
        # initially set as completely random
        self.transition_matrix=generate_state_matrix(
            n_states,random_seed=random_seed
        )

    def update_transition_matrix(self,
                            action):
        """
        
        given the action which is the consequence
        of the interaction between the agent and the 
        player, updates the transition matrix
        """

        current_state=self.states_list[-1]
        updated_transition_matrix=update_transition_matrix(self.transition_matrix, 
                                                           current_state, 
                                                           action)
        
        self.transition_matrix=updated_transition_matrix


    def update_state(self):
        """
        updates the state of the game given the updated 
        transition matrix. 

        done by sampling given the updated
        probability vector.
        """

        current_state=self.states_list[-1]

        self.states_list.append(
            np.random.choice(self.n_states, p=self.transition_matrix[current_state])
        )

        print('The former state was {}. The new state is {}.'.format(
            self.states_list[-1], self.states_list[-1]
        ))

    def terminate(self):
        self.end=(self.states_list[-1]==self.final_state) & (self.states_list[-2]==self.final_state)

    def step(self,action):

        # given the action update the transition matrix
        self.update_transition_matrix(action)

        # and update the state
        self.update_state()

        # update the final condition
        self.terminate()

In [27]:
n_states=3
final_state=0
initial_state=2

test=Architecture(n_states,initial_state,final_state)
print('Initial state is {}'.format(test.initial_state))

print(test.transition_matrix)

action=np.array([2,0,0]) 

for _ in range(10):

    test.step(action)
    print(test.end)



Initial state is 2
[[0.18205878 0.46212909 0.35581214]
 [0.65738127 0.17132261 0.17129612]
 [0.03807826 0.56784481 0.39407693]]
The former state was 1. The new state is 1.
False
The former state was 0. The new state is 0.
False
The former state was 2. The new state is 2.
False
The former state was 0. The new state is 0.
False
The former state was 0. The new state is 0.
True
The former state was 0. The new state is 0.
True
The former state was 0. The new state is 0.
True
The former state was 0. The new state is 0.
True
The former state was 0. The new state is 0.
True
The former state was 0. The new state is 0.
True
